In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use('Qt5Agg')

In [2]:
bids_root = pathlib.Path('out_data/sample_BIDS')

bids_path = mne_bids.BIDSPath(subject = '01',
                             session = '01',
                             task = 'audiovisual',
                             run = '01',
                             datatype = 'meg',
                             root = bids_root)

raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq = 0.1, h_freq = 40)
events, event_id = mne.events_from_annotations(raw)

Opening raw data file out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_meg.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading events from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_events.tsv.
Reading channel info from out_data\sample_BIDS\sub-01\ses-01\meg\sub-01_ses-01_task-audiovisual_run-01_channels.tsv.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


C:\Users\hp\AppData\Roaming\Python\Python37\site-packages\mne_bids\read.py:406: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw.set_channel_types(channel_type_dict)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 19821 samples (33.001 sec)

Used Annotations descriptions: ['Auditory/Left', 'Auditory/Right', 'Button', 'Smiley', 'Visual/Left', 'Visual/Right']


In [3]:
event_id

{'Auditory/Left': 1,
 'Auditory/Right': 2,
 'Button': 3,
 'Smiley': 4,
 'Visual/Left': 5,
 'Visual/Right': 6}

In [10]:
events[:10]

array([[27977,     0,     2],
       [28345,     0,     5],
       [28771,     0,     1],
       [29219,     0,     6],
       [29652,     0,     2],
       [30025,     0,     5],
       [30450,     0,     1],
       [30839,     0,     6],
       [31240,     0,     2],
       [31665,     0,     5]])

Dropped 0 epochs: 
Channels marked as bad: ['MEG 2443', 'EEG 053']
Dropped 0 epochs: 
Channels marked as bad: ['MEG 2443', 'EEG 053']


In [8]:
tmin = -0.3
tmax = 0.5
baseline = (None, 0)

epochs = mne.Epochs(raw,
                   events = events,
                   event_id = event_id,
                   tmin = tmin,
                   tmax = tmax,
                   baseline = baseline,
                   preload = True)

epochs

Not setting metadata
Not setting metadata
320 matching events found
Setting baseline interval to [-0.2996928197375818, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Loading data for 320 events and 481 original time points ...
0 bad epochs dropped


Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Left: 73Visual/Right: 71
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [9]:
epochs.plot()

<MNEBrowseFigure size 1920x932 with 4 Axes>

### Selecting epochs based on experimental conditions

In [11]:
epochs['Auditory/Right']

Number of events,73
Events,Auditory/Right: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [12]:
epochs['Auditory']

Number of events,145
Events,Auditory/Left: 72Auditory/Right: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [13]:
epochs['Left']

Number of events,145
Events,Auditory/Left: 72Visual/Left: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [14]:
epochs['Visual'].plot_image()

Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"
combining channels using "gfp"
combining channels using "gfp"


[<Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>,
 <Figure size 640x476 with 3 Axes>]

In [15]:
(epochs['Right']
 .copy()
 .pick_types(meg = False, eeg = True)
 .plot_image())

## or use:
## epochs['Right'].plot_image(picks = ['eeg'])

Removing projector <Projection | PCA-v1, active : True, n_channels : 102>
Removing projector <Projection | PCA-v2, active : True, n_channels : 102>
Removing projector <Projection | PCA-v3, active : True, n_channels : 102>
Not setting metadata
Not setting metadata
144 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
combining channels using "gfp"


[<Figure size 640x476 with 3 Axes>]

### Saving Epochs

In [16]:
epochs.save(pathlib.Path('out_data') / 'epochs_epo.fif', overwrite = True)

### Creating evoked Data

In [17]:
evoked_auditory = epochs['Auditory'].average()
evoked_visual = epochs['Visual'].average()

In [19]:
evoked_auditory.plot(spatial_colors = True)

<Figure size 640x496 with 6 Axes>

In [23]:
evoked_auditory.plot_topomap(ch_type = 'mag', times = [0, 0.050, 0.100, 0.150, 0.200])

<MNEFigure size 900x220 with 6 Axes>

In [24]:
evoked_auditory.plot_joint(picks = 'mag')

Projections have already been applied. Setting proj attribute to True.


<Figure size 1041x557 with 7 Axes>

In [25]:
mne.viz.plot_compare_evokeds([evoked_auditory, evoked_visual], picks = 'mag')

combining channels using "gfp"
combining channels using "gfp"


[<Figure size 800x596 with 1 Axes>]

### Saving evoked data

In [26]:
mne.write_evokeds(fname = pathlib.Path('out_data') / 'evoked_ave.fif', evoked = [evoked_auditory, evoked_visual])

### Reading evoked data